## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210208.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,400230,16600,416830,378943,0.909107,143552,2021-02-07
1,Aragón,68175,2900,71075,74650,1.050299,24380,2021-02-07
2,Asturias,69935,2900,72835,71784,0.985570,31467,2021-02-07
3,Baleares,35460,1500,36960,35683,0.965449,13430,2021-02-07
4,Canarias,79530,3200,82730,84330,1.019340,28158,2021-02-07
5,Cantabria,33495,1300,34795,34215,0.983331,12427,2021-02-07
6,Castilla y Leon,149475,6200,155675,161020,1.034334,62928,2021-02-07
7,Castilla La Mancha,101655,4200,105855,104988,0.991810,32129,2021-02-07
8,Cataluña,345770,14300,360070,317222,0.881001,120742,2021-02-07
9,C. Valenciana,175690,7000,182690,190902,1.044950,83171,2021-02-07


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                     'Dosis entregadas Moderna','Dosis entregadas',
                     'Dosis administradas','% sobre entregadas',
                     'Total pauta completada',
                     'Fecha de la ultima vacuna registrada',
                     'source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [12]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [13]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [14]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,8/2/21,Andalucía,400230,16600,416830,378943,0.909107,143552,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,8/2/21,Aragón,68175,2900,71075,74650,1.050299,24380,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,8/2/21,Asturias,69935,2900,72835,71784,0.985570,31467,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,8/2/21,Baleares,35460,1500,36960,35683,0.965449,13430,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,8/2/21,Canarias,79530,3200,82730,84330,1.019340,28158,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,8/2/21,Cantabria,33495,1300,34795,34215,0.983331,12427,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,8/2/21,Castilla y Leon,149475,6200,155675,161020,1.034334,62928,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,8/2/21,Castilla La Mancha,101655,4200,105855,104988,0.991810,32129,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,8/2/21,Cataluña,345770,14300,360070,317222,0.881001,120742,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,8/2/21,C. Valenciana,175690,7000,182690,190902,1.044950,83171,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [15]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [16]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [17]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
475,5/2/21,Navarra,33495.0,1300.0,34795,31446,0.903751,9016.0,NaN,4/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
476,5/2/21,País Vasco,91080.0,3600.0,94680,68359,0.722000,21112.0,NaN,4/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
477,5/2/21,Ceuta,1980.0,200.0,2180,2156,0.988991,921.0,NaN,4/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
478,5/2/21,Melilla,1980.0,200.0,2180,2392,1.097248,1073.0,NaN,4/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [18]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [19]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [20]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [21]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
495,8/2/21,Navarra,33495.0,1300.0,34795,34273,0.984998,11439.0,NaN,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
496,8/2/21,País Vasco,91080.0,3600.0,94680,72979,0.770796,24002.0,NaN,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
497,8/2/21,Ceuta,1980.0,200.0,2180,2218,1.017431,964.0,NaN,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
498,8/2/21,Melilla,1980.0,200.0,2180,2398,1.100000,1071.0,NaN,7/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [22]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [23]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)